In [2]:
import itertools

import numpy as np
from scipy import linalg
from scipy.stats import multivariate_normal
from sklearn import mixture
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
color_iter = itertools.cycle(['navy', 'c', 'cornflowerblue', 'gold', 'darkorange'])

def plot_results(X, Y_, means, covariances, index, title):
    splot = plt.subplot(2, 2, 1 + index)
    for i, (mean, covar, color) in enumerate(zip(
            means, covariances, color_iter)):
        v, w = linalg.eigh(covar)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
        plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180. * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        splot.add_artist(ell)

    plt.xlim(-20., 20.)
    plt.ylim(-40., 80.)
    plt.xticks(())
    plt.yticks(())
    plt.title(title)

In [35]:
N_GAUSSIANS = 5

data = np.genfromtxt('sampled_items.csv', delimiter='|')

print data.shape

gmm = mixture.GaussianMixture(n_components=N_GAUSSIANS, max_iter=500).fit(data)

mu = gmm.means_
sigma = gmm.covariances_

print mu.shape
print sigma.shape

(5000, 5)
(5, 5)
(5, 5, 5)


In [36]:
v = (0,1,3,4)
n = data.shape[0]
m = data.shape[1]

t = np.zeros((n, N_GAUSSIANS))
mub = np.zeros((n, m, N_GAUSSIANS))
xb = np.zeros((n, m))

mvns = [multivariate_normal(mu[i, v], sigma[i, v, v]) for i in range(N_GAUSSIANS)]

for i in [0]:
    for k in range(N_GAUSSIANS):
        num = mvns[k].pdf(data[i, v]) 
        den = sum([mvns[pos].pdf(data[i, v]) for pos in range(N_GAUSSIANS)])
            
        t[i, k] = num/den
        for j in range(m):
            all_but_j = np.arange(m) != j
            mub[i, j, k] = mu[k, j] + \
                            np.dot(np.dot(np.delete(sigma[k, j], j), \
                            linalg.pinv(np.delete(sigma[k, all_but_j], j, 1))), \
                            (data[i, all_but_j] - mu[k, all_but_j]))
    
    xb[i, :] = sum([t[i, k]*mub[i, :, k] for k in range(N_GAUSSIANS)])
    print xb[i]
    print data[i]

[  3.20768151e+00   2.15235666e+01   2.41154619e+04   4.83390176e-02
   4.05442284e-02]
[  2.00000000e+00   1.60000000e+01   3.17105600e+04   7.00000000e-02
   4.00000000e-02]
